In [5]:
# Importation des bibliothèques nécessaires
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint  # Import des callbacks




ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [3]:
import os
print(os.getcwd())


/content


In [2]:
import os

# Rechercher le fichier "best_model.keras"
for root, dirs, files in os.walk('/content/drive/MyDrive/ColabNotebooks/'):
    for file in files:
        if file == 'best_model.keras':
            print("Fichier trouvé :", os.path.join(root, file))


In [4]:
# Chargement des données d'entraînement
train_df = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/crops_100K_1/train_100K.csv',sep=r'\s+')
print(train_df.head())
print(train_df.columns)

train_df=train_df.head(5000)


                    filename  FaceOcclusion  gender
0  database1/img00011271.jpg          0.019   0.999
1  database1/img00012471.jpg          0.035   1.000
2  database1/img00008127.jpg          0.127   0.001
3  database1/img00008972.jpg          0.014   0.999
4  database1/img00028187.jpg          0.346   0.982
Index(['filename', 'FaceOcclusion', 'gender'], dtype='object')


In [5]:
from sklearn.model_selection import train_test_split

# Diviser les données : 80% entraînement, 20% validation
train_df, validation_df = train_test_split(
    train_df,
    test_size=0.2,
    random_state=42  # Pour garantir la reproductibilité
)

print(f"Taille des données d'entraînement : {len(train_df)}")
print(f"Taille des données de validation : {len(validation_df)}")


Taille des données d'entraînement : 4000
Taille des données de validation : 1000


In [6]:
import os
print("Répertoire courant :", os.getcwd())
import os
print(os.listdir('/content'))


Répertoire courant : /content
['.config', 'drive', 'sample_data']


In [7]:
import os

# Fonction pour vérifier si un fichier existe
def filter_valid_files(df, base_dir, x_col):
    valid_files = df[x_col].apply(lambda x: os.path.exists(os.path.join(base_dir, x)))
    valid_df = df[valid_files].reset_index(drop=True)
    print(f"Fichiers valides trouvés : {len(valid_df)} / {len(df)}")
    return valid_df

# Filtrer les données pour ne garder que les fichiers valides
base_directory = '/content/drive/MyDrive/ColabNotebooks/crops_100K_1'
train_df = filter_valid_files(train_df, base_directory, 'filename')
validation_df = filter_valid_files(validation_df, base_directory, 'filename')


Fichiers valides trouvés : 4000 / 4000
Fichiers valides trouvés : 1000 / 1000


In [8]:
  from tensorflow.keras.utils import Sequence

  # Définition d'une classe pour sécuriser le générateur
  class SafeDataGenerator(Sequence):
      def __init__(self, generator, **kwargs):
          super().__init__(**kwargs)  # Initialiser la classe parente avec les arguments appropriés
          self.generator = generator

      def __len__(self):
          return len(self.generator)

      def __getitem__(self, idx):
          try:
              return self.generator[idx]
          except Exception as e:
              print(f"Erreur ignorée pour le batch {idx} : {e}")
              return next(iter(self.generator))


  # Définir un générateur d'images avec normalisation
  train_datagen = ImageDataGenerator(
      rescale=1./255,
      validation_split=0.2,  # Séparation pour validation
      horizontal_flip=True,  # Exemple d'augmentation
      rotation_range=10
  )

# Générateur pour l'entraînement
  train_generator = train_datagen.flow_from_dataframe(
      dataframe=train_df,
      directory='/content/drive/MyDrive/ColabNotebooks/crops_100K_1',  # Remplacez par le chemin réel
      x_col='filename',
      y_col='FaceOcclusion',
      subset='training',
      batch_size=32,
      seed=42,
      shuffle=True,
      class_mode='raw',
      target_size=(112, 112)
  )

  # Générateur pour la validation
  validation_generator = train_datagen.flow_from_dataframe(
      dataframe=validation_df,
      directory='/content/drive/MyDrive/ColabNotebooks/crops_100K_1',
      x_col='filename',
      y_col='FaceOcclusion',
      subset='validation',
      batch_size=32,
      seed=42,
      shuffle=False,
      class_mode='raw',
      target_size=(112, 112)
  )

  # Sécuriser les générateurs
  train_generator_safe = SafeDataGenerator(train_generator)
  validation_generator_safe = SafeDataGenerator(validation_generator)


Found 3200 validated image filenames.
Found 200 validated image filenames.


In [9]:
# Définition du modèle CNN optimisé
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(112, 112, 3)),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(1)  # Sortie unique pour la régression
])


In [10]:
# Compilation du modèle avec AdamW
optimizer = optimizers.Adam(learning_rate=1e-4)

model.compile(
    optimizer=optimizer,
    loss='mean_squared_error',
    metrics=['mae']
)

# Affichage du résumé du modèle
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 110, 110, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 110, 110, 32)      128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 108, 108, 32)      9248      
                                                                 
 batch_normalization_1 (Bat  (None, 108, 108, 32)      128       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 54, 54, 32)        0         
 D)                                                              
                                                        

In [ ]:
model.s

In [11]:
# Définition des callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=1)
checkpoint = ModelCheckpoint(filepath='/content/my_best_models/best_model.keras', monitor='val_loss', save_best_only=True, verbose=1)


In [12]:
import os

# Chemin du répertoire à créer
directory_path = '/content/my_best_models'

# Vérifiez si le répertoire existe, sinon créez-le
if not os.path.exists(directory_path):
    os.makedirs(directory_path)
    print(f"Répertoire créé : {directory_path}")
else:
    print(f"Le répertoire existe déjà : {directory_path}")


Répertoire créé : /content/my_best_models


In [13]:
# Entraînement du modèle
history = model.fit(
    train_generator_safe,
    epochs=1,  # Nombre d'époques élevé avec EarlyStopping pour s'arrêter automatiquement
    validation_data=validation_generator_safe,
    callbacks=[early_stop, reduce_lr, checkpoint]
)
model.save(os.path.join(directory_path, 'best_model.keras'))
print(f"Modèle sauvegardé dans : {directory_path}/best_model.keras")



100/100 [==============================] - ETA: 0s - loss: 1.6018 - mae: 0.9515
Epoch 1: val_loss improved from inf to 0.14727, saving model to /content/my_best_models/best_model.keras
100/100 [==============================] - 711s 7s/step - loss: 1.6018 - mae: 0.9515 - val_loss: 0.1473 - val_mae: 0.3700 - lr: 1.0000e-04
Modèle sauvegardé dans : /content/my_best_models/best_model.keras


In [23]:
test_df = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/crops_100K_1/test_students.csv", sep=r'\s+')
test_df.head(10)

,filename
0,database2/database2/test/0.jpg
1,database2/database2/test/1.jpg
2,database2/database2/test/2.jpg
3,database2/database2/test/3.jpg
4,database2/database2/test/4.jpg
5,database2/database2/test/5.jpg
6,database2/database2/test/6.jpg
7,database2/database2/test/7.jpg
8,database2/database2/test/8.jpg
9,database2/database2/test/9.jpg


In [25]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tqdm import tqdm  # Pour afficher une barre de progression lors du chargement des images

# 1. Charger les données de test
data_path = '/content/drive/MyDrive/ColabNotebooks/crops_100K_1/test_students.csv'  # Chemin vers votre fichier CSV de test
test_df = pd.read_csv(data_path)
print("Aperçu des données de test :")
print(test_df.head())
print("\nColonnes disponibles :")
print(test_df.columns)

# Vérifier le nombre de lignes
num_test_samples = test_df.shape[0]
print(f"\nNombre total d'échantillons de test : {num_test_samples}")

# 2. Préparer les données de test

# a. Définir le chemin de base pour les images
base_image_path = '/content/drive/MyDrive/ColabNotebooks/crops_100K_1/'  # Chemin de base pour les images

# b. Définir les paramètres d'image
IMAGE_SIZE = (112, 112)  # Taille d'entrée attendue par le modèle
IMAGE_CHANNELS = 3        # 3 pour RGB, 1 pour grayscale

# c. Fonction pour charger et prétraiter les images
def load_and_preprocess_image(filename, base_path, target_size):
    img_path = os.path.join(base_path, filename)
    try:
        # Charger l'image avec la taille cible correcte
        img = load_img(img_path, target_size=target_size)
        # Convertir en tableau numpy
        img_array = img_to_array(img)
        # Normaliser les pixels entre 0 et 1
        img_array = img_array / 255.0
        return img_array
    except Exception as e:
        print(f"Erreur lors du chargement de l'image {img_path}: {e}")
        # Retourner une image de zéros en cas d'erreur
        return np.zeros((target_size[0], target_size[1], IMAGE_CHANNELS))

# d. Charger et prétraiter toutes les images de test
print("\nChargement et prétraitement des images de test...")
X_test_images = []
for filename in tqdm(test_df['filename'], desc="Chargement des images"):
    img = load_and_preprocess_image(filename, base_image_path, IMAGE_SIZE)
    X_test_images.append(img)
X_test_images = np.array(X_test_images)
print(f"Shape des images de test : {X_test_images.shape}")

# e. (Optionnel) Extraire des caractéristiques supplémentaires
# Si votre modèle utilise des caractéristiques supplémentaires comme 'FaceOcclusion', incluez-les ici.
# Dans cet exemple, nous supposons que seules les images sont utilisées.
# Si 'FaceOcclusion' est nécessaire, décommentez et adaptez le code ci-dessous en conséquence.

# if 'FaceOcclusion' in test_df.columns:
#     X_test_face_occlusion = test_df['FaceOcclusion'].values.reshape(-1, 1)
#     print(f"Shape de FaceOcclusion : {X_test_face_occlusion.shape}")
# else:
#     X_test_face_occlusion = None
#     print("La caractéristique 'FaceOcclusion' n'est pas présente dans les données de test.")

# f. (Optionnel) Préparer les étiquettes
# Puisque le jeu de données de test ne contient pas les étiquettes 'gender', cette section est omise.

# 3. Charger le modèle CNN pré-entraîné
model_path = '/content/my_best_models/best_model.keras'  # Chemin vers votre modèle
model = load_model(model_path)
print("\nModèle chargé avec succès.")
model.summary()

# 4. Préparer les inputs pour le modèle
# Vérifiez les types d'inputs que votre modèle attend
# Dans cet exemple, nous supposons que le modèle prend uniquement les images comme entrée.

# Si votre modèle prend plusieurs entrées (par exemple, images et 'FaceOcclusion'), décommentez et adaptez le code ci-dessous.
# Sinon, passez uniquement les images.

# if isinstance(model.input, list):
#     print(f"\nLe modèle a {len(model.input)} entrées.")
#     # Préparer les inputs en fonction
#     if X_test_face_occlusion is not None:
#         X_test = [X_test_images, X_test_face_occlusion]
#     else:
#         raise ValueError("Le modèle attend plusieurs entrées mais 'FaceOcclusion' n'est pas fournie.")
# else:
#     print("\nLe modèle prend une seule entrée (probablement les images).")
#     X_test = X_test_images

# Dans cet exemple, nous supposons que le modèle prend une seule entrée (les images)
X_test = X_test_images

# 5. Effectuer les prédictions sur les données de test
print("\nEffectuation des prédictions sur les données de test...")
predictions = model.predict(X_test)
print("Prédictions effectuées avec succès.")

# 6. Interpréter les prédictions
# Supposons que c'est une régression pour le pourcentage d'occultations
# Si c'est une classification binaire, ajustez en conséquence.

# Ici, je suppose que le modèle prédit le pourcentage d'occultations directement (régression)
# Si ce n'est pas le cas, veuillez adapter la section ci-dessous en fonction de la sortie de votre modèle.

# Par exemple, si la sortie est une probabilité (0 à 1), multipliez par 100 pour obtenir un pourcentage.
predicted_percentages = predictions.flatten() * 100  # Convertir en pourcentage
predicted_percentages = np.clip(predicted_percentages, 0, 100)  # S'assurer que les valeurs sont entre 0 et 100

# 7. Sauvegarder les résultats dans un fichier texte
# Nom du fichier de sortie
output_filename = 'predictions_nom1_nom2.txt'  # Remplacez 'nom1' et 'nom2' par vos noms

# Ouvrir le fichier en mode écriture
with open(output_filename, 'w') as f:
    for idx, row in test_df.iterrows():
        # Extraire le nom de l'image (par exemple, '0.jpg' à partir de 'database2/database2/test/0.jpg')
        image_path = row['filename']
        image_name = os.path.basename(image_path)
        # Obtenir le pourcentage prédit pour cette image
        percentage = predicted_percentages[idx]
        # Formater le pourcentage avec deux décimales
        percentage_str = f"{percentage:.2f}"
        # Écrire la ligne dans le fichier
        f.write(f"{image_name}.{percentage_str}\n")

print(f"\nLes prédictions ont été sauvegardées dans le fichier : {output_filename}")


Aperçu des données de test :
                         filename
0  database2/database2/test/0.jpg
1  database2/database2/test/1.jpg
2  database2/database2/test/2.jpg
3  database2/database2/test/3.jpg
4  database2/database2/test/4.jpg

Colonnes disponibles :
Index(['filename'], dtype='object')

Nombre total d'échantillons de test : 30507

Chargement et prétraitement des images de test...


Chargement des images: 100%|██████████| 30507/30507 [01:21<00:00, 375.20it/s]


Shape des images de test : (30507, 112, 112, 3)

Modèle chargé avec succès.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 110, 110, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 110, 110, 32)      128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 108, 108, 32)      9248      
                                                                 
 batch_normalization_1 (Bat  (None, 108, 108, 32)      128       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 54, 54, 32)        0         
 D)                                           

In [ ]:







import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tqdm import tqdm  # Pour afficher une barre de progression lors du chargement des images

# 1. Charger les données
data_path = '/content/drive/MyDrive/ColabNotebooks/crops_100K_1/train_100K.csv'
train_df = pd.read_csv(data_path, sep=r'\s+')
print(train_df.head())
print(train_df.columns)

# Optionnel : Réduire le DataFrame pour tester
train_df = train_df.head(5000)

# 2. Diviser les données : 80% entraînement, 20% validation
train_df, validation_df = train_test_split(
    train_df,
    test_size=0.2,
    random_state=42  # Pour garantir la reproductibilité
)

print(f"Taille des données d'entraînement : {len(train_df)}")
print(f"Taille des données de validation : {len(validation_df)}")

# 3. Préparer les données de validation

# a. Définir le chemin de base pour les images
base_image_path = '/content/drive/MyDrive/ColabNotebooks/crops_100K_1/'

# b. Définir les paramètres d'image
IMAGE_SIZE = (112, 112)  # Ajustez selon les besoins de votre modèle
IMAGE_CHANNELS = 3        # 3 pour RGB, 1 pour grayscale

# c. Fonction pour charger et prétraiter les images
def load_and_preprocess_image(filename, base_path, target_size):
    img_path = os.path.join(base_path, filename)
    try:
        # Charger l'image
        img = load_img(img_path, target_size=target_size)
        # Convertir en tableau numpy
        img_array = img_to_array(img)
        # Normaliser les pixels entre 0 et 1
        img_array = img_array / 255.0
        return img_array
    except Exception as e:
        print(f"Erreur lors du chargement de l'image {img_path}: {e}")
        # Retourner une image de zéros en cas d'erreur
        return np.zeros((target_size[0], target_size[1], IMAGE_CHANNELS))

# d. Charger et prétraiter toutes les images de la validation
print("Chargement et prétraitement des images de validation...")
X_val_images = []
for filename in tqdm(validation_df['filename']):
    img = load_and_preprocess_image(filename, base_image_path, IMAGE_SIZE)
    X_val_images.append(img)
X_val_images = np.array(X_val_images)
print(f"Shape des images de validation : {X_val_images.shape}")

X_val_face_occlusion = validation_df['FaceOcclusion'].values.reshape(-1, 1)
print(f"Shape de FaceOcclusion : {X_val_face_occlusion.shape}")

y_val = np.round(validation_df['gender']).astype(int)
print(f"Échantillons d'étiquettes : {y_val[:5]}")

model_path = '/content/my_best_models/best_model.keras'  # Chemin vers votre modèle
model = load_model(model_path)
print("Modèle chargé avec succès.")

model.summary()

if isinstance(model.input, list):
    print(f"Le modèle a {len(model.input)} entrées.")
    X_val = [X_val_images, X_val_face_occlusion]
else:
    print("Le modèle prend une seule entrée (probablement les images).")
    X_val = X_val_images
print("Effectuation des prédictions sur les données de validation...")
predictions = model.predict(X_val)

if predictions.shape[1] == 1:
    predicted_classes = (predictions.flatten() > 0.5).astype(int)
else:
    predicted_classes = np.argmax(predictions, axis=1)

print("Rapport de classification :")
print(classification_report(y_val, predicted_classes))

print("Matrice de confusion :")
print(confusion_matrix(y_val, predicted_classes))

validation_df = validation_df.reset_index(drop=True)  # Réinitialiser les index si nécessaire
validation_df['Predicted_Gender'] = predicted_classes

# Optionnel : Ajouter les probabilités de prédiction
if predictions.shape[1] == 1:
    validation_df['Predicted_Probability'] = predictions.flatten()
else:
    # Pour multi-classes, ajouter les probabilités de chaque classe
    for i, class_name in enumerate(model.classes_ if hasattr(model, 'classes_') else range(predictions.shape[1])):
        validation_df[f'Prob_Class_{i}'] = predictions[:, i]

# Définir le chemin de sortie
output_path = '/content/drive/MyDrive/ColabNotebooks/crops_100K_1/validation_predictions.csv'  # Chemin de sortie

# Sauvegarder dans un fichier CSV
validation_df.to_csv(output_path, index=False)
print(f"Les prédictions ont été sauvegardées dans le fichier : {output_path}")



                    filename  FaceOcclusion  gender
0  database1/img00011271.jpg          0.019   0.999
1  database1/img00012471.jpg          0.035   1.000
2  database1/img00008127.jpg          0.127   0.001
3  database1/img00008972.jpg          0.014   0.999
4  database1/img00028187.jpg          0.346   0.982
Index(['filename', 'FaceOcclusion', 'gender'], dtype='object')
Taille des données d'entraînement : 4000
Taille des données de validation : 1000
Chargement et prétraitement des images de validation...


100%|██████████| 1000/1000 [02:32<00:00,  6.54it/s]


Shape des images de validation : (1000, 224, 224, 3)
Shape de FaceOcclusion : (1000, 1)
Échantillons d'étiquettes : 1501    0
2586    0
2653    0
1055    0
705     0
Name: gender, dtype: int64
Modèle chargé avec succès.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 110, 110, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 110, 110, 32)      128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 108, 108, 32)      9248      
                                                                 
 batch_normalization_1 (Bat  (None, 108, 108, 32)      128       
 chNormalization)                                                
                                  

ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2381, in predict_step
        return self(x, training=False)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueErro

In [17]:

# 7. Préparer les données de test

# a. Charger la liste de test
test_path = '/content/drive/MyDrive/ColabNotebooks/crops_100K_1/test_students.csv'
test_df = pd.read_csv(test_path, sep=r'\s+')
print("Premières lignes du DataFrame de test :")
print(test_df.head())
print("Colonnes du DataFrame de test :", test_df.columns)
print(f"Taille des données de test : {len(test_df)}")

# b. Charger et prétraiter toutes les images de test
print("Chargement et prétraitement des images de test...")
X_test_images = []
for filename in tqdm(test_df['filename']):
    img = load_and_preprocess_image(filename, base_image_path, IMAGE_SIZE)
    X_test_images.append(img)
X_test_images = np.array(X_test_images)
print(f"Shape des images de test : {X_test_images.shape}")

# 8. Effectuer les prédictions sur les données de test
print("Effectuation des prédictions sur les données de test...")
predictions_test = model.predict(X_test_images).flatten()  # Supposant que la sortie est un seul neurone pour la régression

# 9. Générer le fichier de prédictions pour le test
# a. Assurez-vous que les prédictions sont dans le bon ordre
# Le DataFrame test_df est supposé être dans l'ordre correct

# b. Créer le DataFrame de soumission
submission_df = pd.DataFrame({
    'filename': test_df['filename'],
    'FaceOcclusion_Predicted': predictions_test
})

# c. Formater les prédictions pour qu'elles soient arrondies à un certain nombre de décimales (optionnel)
submission_df['FaceOcclusion_Predicted'] = submission_df['FaceOcclusion_Predicted'].round(3)

# d. Sauvegarder dans un fichier texte sans header, séparé par des espaces ou des points comme spécifié
# Selon les spécifications, le format doit être "nom de l’image.pourcentage d’occultations prédit"
# Cela peut être interprété comme "nom de l'image prédiction" séparés par un espace ou un point

# Pour suivre strictement le format "nom de l’image.pourcentage d’occultations prédit",
# nous allons concaténer les colonnes avec un point entre le nom et la prédiction.

submission_df['submission'] = submission_df['filename'] + '.' + submission_df['FaceOcclusion_Predicted'].astype(str)

# Garder uniquement la colonne 'submission'
submission_final = submission_df['submission']

# Définir le nom du fichier de sortie
# Remplacez 'nom1' et 'nom2' par vos noms réels
nom1 = 'Doe'  # Exemple
nom2 = 'Smith'  # Exemple
output_filename = f'predictions_{nom1}_{nom2}.txt'
output_path = os.path.join('/content/drive/MyDrive/ColabNotebooks/crops_100K_1/', output_filename)

# Sauvegarder dans un fichier texte sans index, sans header
submission_final.to_csv(output_path, index=False, header=False)
print(f"Les prédictions sur le jeu de test ont été sauvegardées dans le fichier : {output_path}")

# 10. (Optionnel) Sauvegarder également les prédictions de validation pour analyse
validation_df = validation_df.reset_index(drop=True)  # Réinitialiser les index si nécessaire
validation_df['Predicted_FaceOcclusion'] = predictions_val

# Définir le chemin de sortie pour les prédictions de validation
output_val_path = '/content/drive/MyDrive/ColabNotebooks/crops_100K_1/validation_predictions.csv'
validation_df.to_csv(output_val_path, index=False)

Premières lignes du DataFrame de test :
                         filename
0  database2/database2/test/0.jpg
1  database2/database2/test/1.jpg
2  database2/database2/test/2.jpg
3  database2/database2/test/3.jpg
4  database2/database2/test/4.jpg
Colonnes du DataFrame de test : Index(['filename'], dtype='object')
Taille des données de test : 30507
Chargement et prétraitement des images de test...


100%|██████████| 30507/30507 [1:47:34<00:00,  4.73it/s]


Shape des images de test : (30507, 224, 224, 3)
Effectuation des prédictions sur les données de test...


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2381, in predict_step
        return self(x, training=False)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueErro

In [21]:

import numpy as np

# Sauvegarder l'array en fichier binaire .npy
np.save('x_test_images.npy', X_test_images)
print("DataFrame converti en NumPy array et sauvegardé en 'x_test_images.npy'")


DataFrame converti en NumPy array et sauvegardé en 'x_test_images.npy'


In [22]:
# Afficher le résumé du modèle pour vérifier l'input_shape
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 110, 110, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 110, 110, 32)      128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 108, 108, 32)      9248      
                                                                 
 batch_normalization_1 (Bat  (None, 108, 108, 32)      128       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 54, 54, 32)        0         
 D)                                                              
                                                        

In [ ]:
# Configurer le générateur de données pour le test
test_datagen = ImageDataGenerator(rescale=1./255)
X_test_images=valid_test_df  # Supposant que la sortie est un seul neurone pour la régression

test_generator = test_datagen.flow_from_dataframe(
    dataframe=valid_test_df,
    directory=base_image_dir,
    x_col='filename',
    y_col=None,  # Pas de labels pour le test
    target_size=(112, 112),  # Taille attendue par le modèle
    batch_size=32,
    class_mode=None,  # Pas de labels
    shuffle=False  # Important pour conserver l'ordre des prédictions
)

# 8. Effectuer les prédictions sur les données de test
print("Effectuation des prédictions sur les données de test...")
predictions_test = model.predict(X_test_images).flatten()  # Supposant que la sortie est un seul neurone pour la régression

# 9. Générer le fichier de prédictions pour le test
# a. Assurez-vous que les prédictions sont dans le bon ordre
# Le DataFrame test_df est supposé être dans l'ordre correct

# b. Créer le DataFrame de soumission
submission_df = pd.DataFrame({
    'filename': test_df['filename'],
    'FaceOcclusion_Predicted': predictions_test
})

# c. Formater les prédictions pour qu'elles soient arrondies à un certain nombre de décimales (optionnel)
submission_df['FaceOcclusion_Predicted'] = submission_df['FaceOcclusion_Predicted'].round(3)

# d. Sauvegarder dans un fichier texte sans header, séparé par des espaces ou des points comme spécifié
# Selon les spécifications, le format doit être "nom de l’image.pourcentage d’occultations prédit"
# Cela peut être interprété comme "nom de l'image prédiction" séparés par un espace ou un point

# Pour suivre strictement le format "nom de l’image.pourcentage d’occultations prédit",
# nous allons concaténer les colonnes avec un point entre le nom et la prédiction.

submission_df['submission'] = submission_df['filename'] + '.' + submission_df['FaceOcclusion_Predicted'].astype(str)

# Garder uniquement la colonne 'submission'
submission_final = submission_df['submission']

# Définir le nom du fichier de sortie
# Remplacez 'nom1' et 'nom2' par vos noms réels
nom1 = 'Doe'  # Exemple
nom2 = 'Smith'  # Exemple
output_filename = f'predictions_{nom1}_{nom2}.txt'
output_path = os.path.join('/content/drive/MyDrive/ColabNotebooks/crops_100K_1/', output_filename)

# Sauvegarder dans un fichier texte sans index, sans header
submission_final.to_csv(output_path, index=False, header=False)
print(f"Les prédictions sur le jeu de test ont été sauvegardées dans le fichier : {output_path}")

# 10. (Optionnel) Sauvegarder également les prédictions de validation pour analyse
validation_df = validation_df.reset_index(drop=True)  # Réinitialiser les index si nécessaire
validation_df['Predicted_FaceOcclusion'] = predictions_val

# Définir le chemin de sortie pour les prédictions de validation
output_val_path = '/content/drive/MyDrive/ColabNotebooks/crops_100K_1/validation_predictions.csv'
validation_df.to_csv(output_val_path, index=False)

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import os

# 1. Charger les données d'entraînement
train_data_path = '/content/drive/MyDrive/ColabNotebooks/crops_100K_1/train_100K.csv'  # Remplacez par le chemin de votre fichier CSV d'entraînement
train_df = pd.read_csv(train_data_path,sep='\s+')
print("Aperçu des données d'entraînement :")
print(train_df.head())
print("\nColonnes disponibles :")
print(train_df.columns)

# 2. Diviser les données en ensemble d'entraînement et de validation
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)
print(f"Taille des données d'entraînement : {len(train_df)}")
print(f"Taille des données de validation : {len(val_df)}")

# 3. Préparer les données d'entraînement et de validation

# a. Définir le chemin de base pour les images
base_image_path = '/content/drive/MyDrive/ColabNotebooks/crops_100K_1/'  # Remplacez par le chemin de vos images

# b. Définir les paramètres d'image
IMAGE_SIZE = (299, 299)  # Taille d'entrée attendue par InceptionV3
IMAGE_CHANNELS = 3        # 3 pour RGB

# c. Fonction pour charger et prétraiter les images
def load_and_preprocess_image(filename, base_path, target_size):
    img_path = os.path.join(base_path, filename)
    try:
        img = load_img(img_path, target_size=target_size)
        img_array = img_to_array(img)
        img_array = preprocess_input(img_array)
        return img_array
    except Exception as e:
        print(f"Erreur lors du chargement de l'image {img_path}: {e}")
        return np.zeros((target_size[0], target_size[1], IMAGE_CHANNELS))

# d. Charger et prétraiter les images d'entraînement
print("\nChargement et prétraitement des images d'entraînement...")
X_train_images = []
for filename in tqdm(train_df['filename'], desc="Chargement des images d'entraînement"):
    img = load_and_preprocess_image(filename, base_image_path, IMAGE_SIZE)
    X_train_images.append(img)
X_train_images = np.array(X_train_images)
print(f"Shape des images d'entraînement : {X_train_images.shape}")

# e. Charger et prétraiter les images de validation
print("\nChargement et prétraitement des images de validation...")
X_val_images = []
for filename in tqdm(val_df['filename'], desc="Chargement des images de validation"):
    img = load_and_preprocess_image(filename, base_image_path, IMAGE_SIZE)
    X_val_images.append(img)
X_val_images = np.array(X_val_images)
print(f"Shape des images de validation : {X_val_images.shape}")

# f. Préparer les étiquettes
y_train = train_df['FaceOcclusion'].values
y_val = val_df['FaceOcclusion'].values

# 4. Construire le modèle basé sur InceptionV3 pour la régression
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], IMAGE_CHANNELS))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='linear')(x)  # Régression pour prédire le pourcentage d'occultations
model = Model(inputs=base_model.input, outputs=predictions)

# 5. Geler les couches du modèle de base
for layer in base_model.layers:
    layer.trainable = False

# 6. Compiler le modèle
model.compile(optimizer=Adam(learning_rate=1e-4), loss='mean_squared_error', metrics=['mae'])

# 7. Définir des callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('/content/my_best_models/best_model_inceptionV3.keras', monitor='val_loss', save_best_only=True)
]

# 8. Entraîner le modèle
model.fit(
    X_train_images, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_val_images, y_val),
    callbacks=callbacks,
    verbose=1
)

print("\nEntraînement terminé et modèle sauvegardé.")


Aperçu des données d'entraînement :
                    filename  FaceOcclusion  gender
0  database1/img00011271.jpg          0.019   0.999
1  database1/img00012471.jpg          0.035   1.000
2  database1/img00008127.jpg          0.127   0.001
3  database1/img00008972.jpg          0.014   0.999
4  database1/img00028187.jpg          0.346   0.982

Colonnes disponibles :
Index(['filename', 'FaceOcclusion', 'gender'], dtype='object')
Taille des données d'entraînement : 81076
Taille des données de validation : 20269

Chargement et prétraitement des images d'entraînement...


Chargement des images d'entraînement:   0%|          | 42/81076 [00:09<5:13:50,  4.30it/s]

Erreur lors du chargement de l'image /content/drive/MyDrive/ColabNotebooks/crops_100K_1/database3/database3/m.01d30x/39-FaceId-0_align.jpg: [Errno 2] No such file or directory: '/content/drive/MyDrive/ColabNotebooks/crops_100K_1/database3/database3/m.01d30x/39-FaceId-0_align.jpg'


Chargement des images d'entraînement:   0%|          | 66/81076 [00:13<3:04:43,  7.31it/s]

Erreur lors du chargement de l'image /content/drive/MyDrive/ColabNotebooks/crops_100K_1/database3/database3/m.01dgtv/44-FaceId-0_align.jpg: [Errno 2] No such file or directory: '/content/drive/MyDrive/ColabNotebooks/crops_100K_1/database3/database3/m.01dgtv/44-FaceId-0_align.jpg'


...

Chargement des images de validation:  36%|███▌      | 7331/20269 [25:16<28:46,  7.50it/s]

Erreur lors du chargement de l'image /content/drive/MyDrive/ColabNotebooks/crops_100K_1/database3/database3/m.01ddnt/39-FaceId-0_align.jpg: [Errno 2] No such file or directory: '/content/drive/MyDrive/ColabNotebooks/crops_100K_1/database3/database3/m.01ddnt/39-FaceId-0_align.jpg'
Erreur lors du chargement de l'image /content/drive/MyDrive/ColabNotebooks/crops_100K_1/database3/database3/m.01d0fp/101-FaceId-0_align.jpg: [Errno 2] No such file or directory: '/content/drive/MyDrive/ColabNotebooks/crops_100K_1/database3/database3/m.01d0fp/101-FaceId-0_align.jpg'
Erreur lors du chargement de l'image /content/drive/MyDrive/ColabNotebooks/crops_100K_1/database3/database3/m.01d4cb/10-FaceId-0_align.jpg: [Errno 2] No such file or directory: '/content/drive/MyDrive/ColabNotebooks/crops_100K_1/database3/database3/m.01d4cb/10-FaceId-0_align.jpg'
Erreur lors du chargement de l'image /content/drive/MyDrive/ColabNotebooks/crops_100K_1/database3/database3/m.01dpsv/128-FaceId-0_align.jpg: [Errno 2] No s

Chargement des images de validation:  36%|███▌      | 7347/20269 [25:19<47:15,  4.56it/s]

Erreur lors du chargement de l'image /content/drive/MyDrive/ColabNotebooks/crops_100K_1/database3/database3/m.01dlw6/4-FaceId-0_align.jpg: [Errno 2] No such file or directory: '/content/drive/MyDrive/ColabNotebooks/crops_100K_1/database3/database3/m.01dlw6/4-FaceId-0_align.jpg'


Chargement des images de validation:  36%|███▋      | 7356/20269 [25:21<46:10,  4.66it/s]

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tqdm import tqdm
import os

# 1. Charger les données de test
data_path = '/content/drive/MyDrive/ColabNotebooks/crops_100K_1/test_student.csv'  # Remplacez par le chemin de votre fichier CSV de test
test_df = pd.read_csv(data_path)
print("Aperçu des données de test :")
print(test_df.head())
print("\nColonnes disponibles :")
print(test_df.columns)

# Vérifier le nombre de lignes
num_test_samples = test_df.shape[0]
print(f"\nNombre total d'échantillons de test : {num_test_samples}")

# 2. Préparer les données de test

# a. Définir le chemin de base pour les images
base_image_path = '/content/drive/MyDrive/ColabNotebooks/crops_100K_1/'  # Remplacez par le chemin de vos images

# b. Définir les paramètres d'image
IMAGE_SIZE = (299, 299)  # Taille d'entrée attendue par InceptionV3
IMAGE_CHANNELS = 3        # 3 pour RGB

# c. Fonction pour charger et prétraiter les images
def load_and_preprocess_image(filename, base_path, target_size):
    img_path = os.path.join(base_path, filename)
    try:
        img = load_img(img_path, target_size=target_size)
        img_array = img_to_array(img)
        img_array = preprocess_input(img_array)
        return img_array
    except Exception as e:
        print(f"Erreur lors du chargement de l'image {img_path}: {e}")
        return np.zeros((target_size[0], target_size[1], IMAGE_CHANNELS))

# d. Charger et prétraiter toutes les images de test
print("\nChargement et prétraitement des images de test...")
X_test_images = []
for filename in tqdm(test_df['filename'], desc="Chargement des images"):
    img = load_and_preprocess_image(filename, base_image_path, IMAGE_SIZE)
    X_test_images.append(img)
X_test_images = np.array(X_test_images)
print(f"Shape des images de test : {X_test_images.shape}")

# 3. Charger le modèle InceptionV3 pré-entraîné avec des couches personnalisées pour la régression
model_path = '/content/my_best_models/best_model_inceptionV3.keras'  # Remplacez par le chemin vers votre modèle InceptionV3 pré-entraîné

if os.path.exists(model_path):
    # Charger le modèle pré-entraîné
    model = load_model(model_path)
    print("\nModèle InceptionV3 chargé avec succès.")
else:
    raise FileNotFoundError(f"Le modèle spécifié n'existe pas à l'emplacement : {model_path}")

# 4. Préparer les inputs pour le modèle
X_test = X_test_images

# 5. Effectuer les prédictions sur les données de test
print("\nEffectuation des prédictions sur les données de test...")
predictions = model.predict(X_test, batch_size=32, verbose=1)
print("Prédictions effectuées avec succès.")

# 6. Interpréter les prédictions
# Supposons que le modèle prédit directement le pourcentage d'occultations (régression)
predicted_percentages = predictions.flatten()  # Réduire les dimensions
predicted_percentages = np.clip(predicted_percentages, 0, 100)  # S'assurer que les valeurs sont entre 0 et 100

# 7. Sauvegarder les résultats dans un fichier texte
# Nom du fichier de sortie (remplacez 'nom1' et 'nom2' par vos noms)
nom1 = 'Alice'  # Remplacez par votre premier nom
nom2 = 'Bob'    # Remplacez par votre deuxième nom
output_filename = f'predictions_{nom1}_{nom2}.txt'  # Exemple : 'predictions_Alice_Bob.txt'

# Ouvrir le fichier en mode écriture
with open(output_filename, 'w') as f:
    for idx, row in test_df.iterrows():
        # Extraire le nom de l'image (par exemple, 'img00011271.jpg' à partir de 'database1/img00011271.jpg')
        image_path = row['filename']
        image_name = os.path.basename(image_path)
        # Obtenir le pourcentage prédit pour cette image
        percentage = predicted_percentages[idx]
        # Formater le pourcentage avec deux décimales
        percentage_str = f"{percentage:.2f}"
        # Écrire la ligne dans le fichier
        f.write(f"{image_name}.{percentage_str}\n")

print(f"\nLes prédictions ont été sauvegardées dans le fichier : {output_filename}")
